In [31]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
torch.cuda.is_available()

In [ ]:
import torch.nn as nn

In [ ]:
x = torch.randn(1,10)

In [ ]:
x

In [ ]:
x._values()

In [ ]:
import tensorflow as tf

In [ ]:
num_nonzero_elems = 5
noise_shape = [num_nonzero_elems]

In [ ]:
keep_prob = 0.7
random_tensor = keep_prob
temp = tf.random.uniform(noise_shape)
print(temp)
random_tensor += temp
print(random_tensor)

In [ ]:
random_tensor

In [ ]:
import pandas as pd
import numpy as np

In [17]:
dataFile = pd.read_csv('../data/full_context_PeerRead.csv')

In [18]:
dataFile.head(100)


,right_citated_text,left_citated_text,source_abstract,source_author,source_id,source_title,source_venue,source_year,target_id,target_author,target_abstract,target_year,target_title,target_venue
0,andsyntactic parsing .Because RNNs make very f...,We conducted additional experiments on artific...,Deep Neural Networks (DNNs) are powerful model...,ilya sutskever;oriol vinyals;quoc v le,1409.3215v1,Sequence to Sequence Learning with Neural Netw...,NIPS,2014.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
1,.Because RNNs make very few domain-specific as...,We conducted additional experiments on artific...,Syntactic parsing is a fundamental problem in ...,oriol vinyals;lukasz kaiser;terry koo;slav pet...,1412.7449v1,Grammar as a Foreign Language,NIPS,2014.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
2,"; in a Pointer Network,the only way to generat...","Reproducibility. All code, data, and experimen...",We introduce a new neural architecture to lear...,oriol vinyals;meire fortunato;navdeep jaitly,1506.03134v1,Pointer Networks,NIPS,2015.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
3,". Recently, nsur . have shown superior perfor...","st like CWS and POS tagging, automatic prosody...",The recently introduced continuous Skip-gram m...,tomas mikolov;ilya sutskever;kai chen 0010;gre...,1310.4546v1,Distributed Representations of Words and Phras...,NIPS,2013.0,1511.00360v1,chuang ding;lei xie;jie yan;weini zhang;yang liu,Prosody affects the naturalness and intelligib...,2015,Automatic Prosody Prediction for Chinese Speec...,arxiv
4,model trained on the Google News dataset3.In a...,We begin by considering a document as the set ...,The recently introduced continuous Skip-gram m...,tomas mikolov;ilya sutskever;kai chen 0010;gre...,1310.4546v1,Distributed Representations of Words and Phras...,NIPS,2013.0,1705.10900v1,paul michel;abhilasha ravichander;shruti rijhwani,We investigate the pertinence of methods from ...,2017,Does the Geometry of Word Embeddings Help Docu...,arxiv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"to create a sentence embedding. Second, from e...",For each word in the sentence we calculate var...,In this paper we compare different types of re...,junyoung chung;caglar gulcehre;kyunghyun cho;y...,1412.3555v1,Empirical Evaluation of Gated Recurrent Neural...,arxiv,2014.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
96,for accelerating training. The network is opti...,For each Q-R pair we extract two sets of featu...,Training Deep Neural Networks is complicated b...,sergey ioffe;christian szegedy,1502.03167v1,Batch Normalization: Accelerating Deep Network...,ICML,2015.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
97,optimizer with learning rate of 0.001. We have...,For each Q-R pair we extract two sets of featu...,"We introduce Adam, an algorithm for first-orde...",diederik p kingma;jimmy ba,1412.6980v1,Adam: A Method for Stochastic Optimization,iclr,2014.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
98,", and convolutional neural networks have show...",Modeling textual or visual information with ve...,Deep Neural Networks (DNNs) are powerful model...,ilya sutskever;oriol vinyals;quoc v le,1409.3215v1,Sequence to Sequence Learning with Neural Netw...,NIPS,2014.0,1606.01847v1,akira fukui;dong huk park;daylen yang;anna roh...,Modeling textual or visual information with ve...,2016,Multimodal Compa

In [21]:
column = ['left_citated_text', 'right_citated_text', 'target_id', 'source_id', 'target_year','target_author']
frequency = 5

In [22]:
df = dataFile[column]
source_cut_data = df[['target_id', 'source_id']].drop_duplicates(subset=['target_id', 'source_id'])
source_cut = source_cut_data.source_id.value_counts()[(source_cut_data.source_id.value_counts() >= frequency)]

In [ ]:
source_id = np.sort(source_cut.keys())
source_id
df = df.loc[df['source_id'].isin(source_id)]

In [ ]:
source_id

In [ ]:
df

In [ ]:
df['#1 String'] = df['left_citated_text'].str[-128:]
df['#2 String'] = df['right_citated_text'].str[:128]
df

In [ ]:
left_citated_text  = df['#1 String'].values.tolist()
right_citated_text = df['#2 String'].values.tolist()
total_citated_text = list(set(left_citated_text + right_citated_text))
citated_voca = {}
left_citated_id = []
right_citated_id = []
for i, v in enumerate(total_citated_text):
    citated_voca[v] = i
citated_voca

In [ ]:
for l, r in zip(df['#1 String'], df['#2 String']):
        left_citated_id.append(citated_voca[l])
        right_citated_id.append(citated_voca[r])

In [ ]:
df['#1 ID'] = left_citated_id
df['#2 ID'] = right_citated_id

In [ ]:
df

In [ ]:
year = 2017
train_idx = df['target_year'][df['target_year'] < year].index
test_idx = df['target_year'][df['target_year'] >= year].index
train_df = df.loc[train_idx]
test_df = df.loc[test_idx]

In [ ]:
test_df

In [ ]:
bert_column = ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String', 'target_id']

In [23]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit_transform(df['source_id'].values)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
lb.get_params()

{'neg_label': 0, 'pos_label': 1, 'sparse_output': False}

In [ ]:
def convert_argmax(df, lb):

    y = df['source_id'].values
    y = lb.transform(y)
    y = np.argmax(y, axis=1)
    df['Quality'] = y

    return df

In [ ]:
train_df1 = convert_argmax(train_df, lb)

In [ ]:
train_df1

In [ ]:
test_df1 = convert_argmax(test_df, lb)

In [ ]:
test_df1

In [ ]:
train_df, test_df = train_df1[bert_column], test_df1[bert_column]

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
train_df.loc[train_df['#1 ID'] == 14050]

In [ ]:
temp_df = test_df

In [ ]:
temp_df.reset_index()

In [ ]:
multi_label_info = []
for i in temp_df.groupby(['#1 ID', '#2 ID']):
    instance_label = {}
    instance_label['Quality'] = i[1]['Quality'].values
    instance_label['index'] = i[1]['Quality'].index.values
    multi_label_info.append(instance_label)

In [ ]:
for i in range (0,len(multi_label_info)):
    if multi_label_info[i]['index'].any() == 0:
        print(multi_label_info[i])
        print(i)

In [ ]:
multi_label_info[0]['index']

In [ ]:
import tensorflow as tf
print("Num GPUs Available: HI ", len(tf.config.experimental.list_physical_devices('GPU')))

### Preprocessing

In [1]:
import pandas as pd
import transformers
import torch
import tqdm

In [3]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [13]:
tokenizer.encode_plus("You are using pip version.","Just like a watched pot never boils, a watched for loop never ends. When dealing with large datasets, even the simplest operations can take hours. Progress bars can help make data processing jobs less of a headache because:",  add_special_tokens = True, max_length = 100, padding=True)

{'input_ids': [101, 2017, 2024, 2478, 28315, 2544, 1012, 102, 2074, 2066, 1037, 3427, 8962, 2196, 26077, 2015, 1010, 1037, 3427, 2005, 7077, 2196, 4515, 1012, 2043, 7149, 2007, 2312, 2951, 13462, 2015, 1010, 2130, 1996, 21304, 3136, 2064, 2202, 2847, 1012, 5082, 6963, 2064, 2393, 2191, 2951, 6364, 5841, 2625, 1997, 1037, 14978, 2138, 1024, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenizer.encode_plus("You are using 117 pip version. Just like a watched pot never boils, a watched for loop never ends. When dealing with large datasets, even the simplest operations can take hours. Progress bars can help make data processing jobs less of a headache because:", None, add_special_tokens = False, max_length = 100, padding=True)

{'input_ids': [2017, 2024, 2478, 12567, 28315, 2544, 1012, 2074, 2066, 1037, 3427, 8962, 2196, 26077, 2015, 1010, 1037, 3427, 2005, 7077, 2196, 4515, 1012, 2043, 7149, 2007, 2312, 2951, 13462, 2015, 1010, 2130, 1996, 21304, 3136, 2064, 2202, 2847, 1012, 5082, 6963, 2064, 2393, 2191, 2951, 6364, 5841, 2625, 1997, 1037, 14978, 2138, 1024], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 101)

In [15]:
context = "You are using pip version.Just like a watched pot never boils"
print(" ".join(context.split()))

You are using pip version.Just like a watched pot never boils


In [20]:
dataFile["right_citated_text"].values[0]

"andsyntactic parsing .Because RNNs make very few domain-specific assumptions,they have the potential to succeed at a wide variety of taskswith minimal feature engineering.wever, this flexibility also puts RNNs at a disadvantage compared tostandard semantic parsers, which can generalize naturallyby leveraging their built-in awareness of logical compositionality.In this paper, we introduce data recombination,a generic framework for declaratively injecting prior knowledgeinto a domain-general structured prediction model.In data recombination, prior knowledge about a taskis used to build a high-precision generative modelthat expands the empirical distributionby allowing fragments of different examples to be combined inparticular ways.mples from this generative model are then used to train a domain-general model.In the case of semantic parsing, we construct a generative modelby inducing a synchronous context-free grammar ,creating new examples such as those shown in Figure 1;our domain-gen